In [ ]:
import sys
print(sys.version)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://apis.data.go.kr/1170000/law/lawSearchList.do?serviceKey=BAMeC3FzuB5pthgorHym0lfi5omDMuquVLDIWwoteNaPfAQ4Jsk8lqDRZCReamd9PyVkW3ww%2BasXUKdJybHxrA%3D%3D&target=law&query=*&numOfRows=10000000&pageNo=1'

df11 = pd.read_xml(url, xpath = './/law')
df11.head()

# 선거와 투표 관련된 법령들 목록 리스트로 저장

In [ ]:
df1 = df11.query('법령명한글.str.contains("선거|투표")',engine='python')

list10=list(df1['법령일련번호'].unique())
l1=list(df11['법령ID'].unique())

In [ ]:
df1.to_csv('./votelist.csv', index=False)

# 앞에서 만든 리스트 기반으로 조항,항제,호제,목제 추출
    조항,항제,호제,목제 폴더에 법령ID로 분류해서 따로 저장

# 조문

In [151]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import tqdm

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")

#조문단위에서 뽑아올 XML요소 정리
contents = ['조문번호','조문가지번호','조문여부','조문제목','조문시행일자',
            '조문이동이전','조문이동이후','조문변경여부','조문제개정유형',
            '조문내용','조문참고자료','항']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        #각 법령에 접근하기 위해 URL을 지정해주는 부분
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        #print(url)
        #print(xtree[1][0].attrib['조문키'])

        # 각 법령에 접근 후 조문 읽어오기
        # len(xtree[1]) 은 조문단위의 수 만큼 접근
        for a in range(len(xtree[1])):
            sub_dict['법령ID'] = xtree[0][0].text
            sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
            # contents는 조문단위의 구성 XML 요소들
            for content in contents:
                dict_key = content
                try:
                    dict_value = xtree[1][a].find(content).text
                except:
                    dict_value = ' '
                sub_dict[dict_key] = dict_value
            rows.append(sub_dict)    
            sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './조항_법령ID/{0}.csv'.format(file_name1)

        each_law_content.to_csv(path_format1, index=False)
        
        rows = []
    except:
        continue

100%|██████████| 50/50 [01:08<00:00,  1.38s/it]


# 법령 항제

In [152]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import re

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")
#조문단위에서 뽑아올 XML요소 정리
contents = ['항번호','항제개정유형','항제개정일자문자열','항내용','호']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        #각 법령에 접근하기 위해 URL을 지정해주는 부분
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        for a in range(len(xtree[1])):
            for b in range(len(xtree[1][a])):
                sub_dict['법령ID'] = xtree[0][0].text
                sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
                for content in contents:
                    dict_key = content
                    try:
                        dict_value = xtree[1][a][b].find(content).text
                    except:
                        dict_value = ' '
                    sub_dict[dict_key] = dict_value
                rows.append(sub_dict)    
                sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './항제_법령ID/{0}.csv'.format(file_name1)

        each_law_content = pd.DataFrame(rows)
        each_law_content.to_csv(path_format1, index=False, encoding='utf-8')

        rows = []
    except:
        continue

100%|██████████| 50/50 [01:08<00:00,  1.36s/it]


# 호제

In [204]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import re

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")

#조문단위에서 뽑아올 XML요소 정리
contents = ['호번호','호내용','목']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        for a in range(len(xtree[1])):
            for b in range(len(xtree[1][a])):
                for c in range(len(xtree[1][a][b])):
                    sub_dict['법령ID'] = xtree[0][0].text
                    sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
                    try:
                        sub_dict['항번호'] = xtree[1][a][b].find('항번호').text
                    except:
                        sub_dict['항번호'] = ' '
                    for content in contents:
                        dict_key = content
                        try:
                            dict_value = xtree[1][a][b][c].find(content).text
                        except:
                            dict_value = ' '
                        sub_dict[dict_key] = dict_value
                    rows.append(sub_dict)    
                    sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './호제_법령ID/{0}.csv'.format(file_name1)

        each_law_content = pd.DataFrame(rows)
        each_law_content.to_csv(path_format1, index=False, encoding='utf-8')

        rows = []
    except:
        continue

100%|██████████| 50/50 [01:07<00:00,  1.35s/it]


# 목제

In [205]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import re

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")

#조문단위에서 뽑아올 XML요소 정리
contents = ['목번호','목내용']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        for a in range(len(xtree[1])):
            for b in range(len(xtree[1][a])):
                for c in range(len(xtree[1][a][b])):
                    for d in range(len(xtree[1][a][b][c])):
                        sub_dict['법령ID'] = xtree[0][0].text
                        sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
                        try:
                            sub_dict['항번호'] = xtree[1][a][b].find('항번호').text
                        except:
                            sub_dict['항번호'] = ' '
                        try:
                            sub_dict['호번호'] = xtree[1][a][b][c].find('호번호').text
                        except:
                            sub_dict['호번호'] = ' '
                        for content in contents:
                            dict_key = content
                            try:
                                dict_value = xtree[1][a][b][c][d].find(content).text
                            except:
                                dict_value = ' '
                            sub_dict[dict_key] = dict_value
                        rows.append(sub_dict)    
                        sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './목제_법령ID/{0}.csv'.format(file_name1)

        each_law_content = pd.DataFrame(rows)
        each_law_content.to_csv(path_format1, index=False, encoding='utf-8')

        rows = []
    except:
        continue

100%|██████████| 50/50 [01:14<00:00,  1.48s/it]


# 데이터 합치기

In [7]:
import os
files = os.listdir('./조항_법령ID')
for i in files:
    globals()['조항{}'.format(i.split('.')[0])] = pd.read_csv(f'./조항_법령ID/{i}')
   


In [8]:
a = [조항001399, 조항001723, 조항001725, 조항002847, 조항006026, 조항006032, 조항006033, 조항006034, 조항006035, 조항006037, 조항006042, 조항006043, 조항006044, 조항006045, 조항006046, 
     조항006047, 조항006049, 조항006050, 조항006054, 조항006057, 조항006064, 조항006065, 조항006069, 
     조항009364, 조항009507, 조항009615, 조항009617, 조항009629, 조항009729, 조항009796, 조항009797, 조항009919, 조항010174, 조항010351, 조항010367, 조항010397, 조항010920, 조항011102, 조항011704, 
     조항011968, 조항011969, 조항011991, 조항012010, 조항012073, 조항012113, 조항012319,
     조항013588, 조항014206, 조항014264, 조항014303]


In [9]:
files = os.listdir('./호제_법령ID')

for i in files:
    globals()['호제{}'.format(i.split('.')[0])] = pd.read_csv(f'./호제_법령ID/{i}')
    


In [10]:
b = [호제001399, 호제001723, 호제001725, 호제002847, 호제006026, 호제006032, 호제006033, 호제006034, 호제006035, 호제006037, 호제006042, 호제006043, 호제006044, 호제006045, 호제006046, 
     호제006047, 호제006049, 호제006050, 호제006054, 호제006057, 호제006064, 호제006065, 호제006069, 
     호제009364, 호제009507, 호제009615, 호제009617, 호제009629, 호제009729, 호제009796, 호제009797, 호제009919, 호제010174, 호제010351, 호제010367, 호제010397, 호제010920, 호제011102, 호제011704, 
     호제011968, 호제011969, 호제011991, 호제012010, 호제012073, 호제012113, 호제012319,
     호제013588, 호제014206, 호제014264, 호제014303]

In [11]:
files = os.listdir('./항제_법령ID')

for i in files:
    globals()['항제{}'.format(i.split('.')[0])] = pd.read_csv(f'./항제_법령ID/{i}')

In [12]:
c = [항제001399, 항제001723, 항제001725, 항제002847, 항제006026, 항제006032, 항제006033, 항제006034, 항제006035, 항제006037, 항제006042, 항제006043, 항제006044, 항제006045, 항제006046, 
     항제006047, 항제006049, 항제006050, 항제006054, 항제006057, 항제006064, 항제006065, 항제006069, 
     항제009364, 항제009507, 항제009615, 항제009617, 항제009629, 항제009729, 항제009796, 항제009797, 항제009919, 항제010174, 항제010351, 항제010367, 항제010397, 항제010920, 항제011102, 항제011704, 
     항제011968, 항제011969, 항제011991, 항제012010, 항제012073, 항제012113, 항제012319,
     항제013588, 항제014206, 항제014264, 항제014303]

In [39]:
for i,y in zip(range(len(files)),files):
    total1 = pd.merge(a[i], c[i], how='outer',on='조문키')
    total1 = pd.merge(total1, b[i], how='outer', on='조문키')
    total1.drop(['조문가지번호', '조문이동이전', '조문이동이후','조문변경여부', '조문제개정유형', '호', '호번호',  '목','항번호_x','항번호_y','법령ID_x','법령ID_y','항'],axis=1,inplace=True)
    total1 = total1.drop_duplicates().reset_index(drop=True)
    
    path = './total/{0}.csv'.format(y.split('.')[0])
    total1.to_csv(path, index = None,encoding='utf-8') #csv파일로 생성